In [3]:
import re
import json
import random
import string
from typing import Dict, List, Tuple, Optional
from datetime import datetime
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class TextPreprocessor:
    """Custom text preprocessing without spaCy"""
    
    def __init__(self):
        # Common English stopwords
        self.stop_words = {
            'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", 
            "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 
            'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 
            'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
            'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 
            'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 
            'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
            'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
            'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 
            'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
            'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
            'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', 
            "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 
            "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', 
            "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 
            "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
        }
        
    def preprocess_text(self, text: str) -> str:
        """Clean and preprocess text: lowercase, remove punctuation, stopwords, and lemmatize"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Tokenize
        tokens = text.split()
        
        # Remove stopwords and short tokens
        filtered_tokens = [
            self._simple_stem(token) for token in tokens 
            if token not in self.stop_words and len(token) > 2
        ]
        
        return ' '.join(filtered_tokens)
    
    def _simple_stem(self, word: str) -> str:
        """Basic stemming - remove common suffixes"""
        if len(word) <= 3:
            return word
            
        # Common English suffixes
        suffixes = ['ing', 'ed', 'es', 's', 'ly']
        for suffix in suffixes:
            if word.endswith(suffix) and len(word) > len(suffix) + 1:
                return word[:-len(suffix)]
        return word
    
    def extract_entities(self, text: str) -> Dict[str, List[str]]:
        """Extract order numbers and product names using regex and keyword matching"""
        entities = {
            "order_number": [],
            "product_name": []
        }
        
        # Extract order numbers (5+ digits, optionally prefixed with #)
        order_numbers = re.findall(r'#?(\d{5,})', text)
        entities["order_number"] = order_numbers
        
        # Extract product names using keyword matching
        product_keywords = {
            'phone': ['phone', 'mobile', 'cellphone', 'smartphone'],
            'laptop': ['laptop', 'notebook', 'computer', 'macbook'],
            'headphones': ['headphones', 'earphones', 'headset', 'earbuds'],
            'tablet': ['tablet', 'ipad', 'surface'],
            'camera': ['camera', 'dslr', 'digital camera'],
            'watch': ['watch', 'smartwatch', 'wristwatch']
        }
        
        words = text.lower().split()
        for word in words:
            for product, keywords in product_keywords.items():
                if word in keywords and product not in entities["product_name"]:
                    entities["product_name"].append(product)
        
        return entities

class ChatbotData:
    def __init__(self):
        self.intents = {
            "order_status": {
                "patterns": [
                    "where is my order", "order status", "track my order",
                    "when will my order arrive", "order tracking",
                    "what is the status of my order", "order number",
                    "track order", "check order status", "my order"
                ],
                "responses": [
                    "Your order {order_number} is currently {status}. Estimated delivery: {estimated_delivery}",
                    "I've checked your order {order_number} - it's {status}. Expected delivery: {estimated_delivery}",
                    "The status of order {order_number} is: {status}. Delivery estimate: {estimated_delivery}"
                ],
                "entities": ["order_number"]
            },
            "return_policy": {
                "patterns": [
                    "how to return", "return policy", "want to return",
                    "return a product", "refund policy", "send back",
                    "how do I return", "return item", "get refund",
                    "return process", "send item back"
                ],
                "responses": [
                    "You can return products within 15 days via our online portal. Items must be unused and in original packaging.",
                    "Returns are accepted within 15 days of delivery through our portal. Please ensure items are in original condition.",
                    "Our return policy allows returns within 15 days using our online system. Shipping costs are non-refundable."
                ]
            },
            "delivery_time": {
                "patterns": [
                    "delivery time", "when will it arrive", "shipping time",
                    "how long for delivery", "delivery estimate", "shipping duration",
                    "when delivery", "how long shipping", "delivery schedule"
                ],
                "responses": [
                    "Standard delivery takes 3-5 business days, express takes 1-2 days.",
                    "Delivery typically takes 3-5 days for standard shipping. Express options available for faster delivery.",
                    "You can expect delivery in 3-5 business days for standard shipping. Express shipping delivers in 1-2 days."
                ]
            },
            "product_info": {
                "patterns": [
                    "does this support", "product features", "specifications",
                    "what are the features", "product details", "fast charging",
                    "battery life", "warranty", "product information",
                    "tell me about product", "what features", "product specs"
                ],
                "responses": [
                    "This product supports fast charging and comes with a 1-year warranty.",
                    "The product features fast charging capability and 1-year warranty.",
                    "Yes, it includes fast charging and has a 1-year warranty period."
                ]
            },
            "cancel_order": {
                "patterns": [
                    "cancel order", "want to cancel", "stop my order",
                    "cancel my purchase", "need to cancel", "cancel item",
                    "how to cancel order", "order cancellation"
                ],
                "responses": [
                    "You can cancel your order within 1 hour of placement through your account page.",
                    "Orders can be cancelled within 1 hour via your account dashboard. After that, contact support.",
                    "Cancellations are possible within 1 hour of ordering from your account section."
                ]
            },
            "contact_support": {
                "patterns": [
                    "talk to human", "live agent", "customer service",
                    "contact support", "speak to representative", "real person",
                    "human help", "customer support", "help desk"
                ],
                "responses": [
                    "Our customer service team is available at 1-800-123-4567 from 9 AM to 6 PM EST.",
                    "You can reach our support team at support@company.com or call 1-800-123-4567.",
                    "For immediate assistance, please call our support line at 1-800-123-4567."
                ]
            },
            "greeting": {
                "patterns": ["hello", "hi", "hey", "good morning", "good afternoon", "greetings"],
                "responses": [
                    "Hello! How can I help you with your order today?",
                    "Hi there! What can I assist you with?",
                    "Welcome! How can I help you today?"
                ]
            },
            "thanks": {
                "patterns": ["thank you", "thanks", "appreciate it", "thank you very much", "thanks a lot"],
                "responses": [
                    "You're welcome! Is there anything else I can help with?",
                    "Happy to help! Let me know if you need anything else.",
                    "Glad I could assist! Feel free to ask if you have more questions."
                ]
            }
        }
        
        # Mock order database
        self.order_database = {
            "12345": {"status": "out for delivery", "estimated_delivery": "tomorrow"},
            "67890": {"status": "processing", "estimated_delivery": "3-5 business days"},
            "11121": {"status": "shipped", "estimated_delivery": "2-3 business days"},
            "31415": {"status": "delivered", "estimated_delivery": "delivered on 2024-01-15"},
            "92653": {"status": "ready for pickup", "estimated_delivery": "ready for pickup"}
        }
        
        # Product database
        self.product_database = {
            "phone": "Supports fast charging, 128GB storage, 5G capable, 1-year warranty",
            "laptop": "16GB RAM, 512GB SSD, Intel i7 processor, 2-year warranty",
            "headphones": "Wireless, noise-cancelling, 30-hour battery life, 1-year warranty",
            "tablet": "10-inch display, 64GB storage, supports stylus, 1-year warranty",
            "camera": "24MP sensor, 4K video, WiFi connectivity, 1-year warranty",
            "watch": "Smartwatch, heart rate monitor, GPS, 2-day battery life"
        }

    def get_order_status(self, order_number: str) -> dict:
        return self.order_database.get(order_number, {"status": "not found", "estimated_delivery": "N/A"})

    def get_product_info(self, product_name: str) -> str:
        return self.product_database.get(product_name.lower(), "Product information not available")

class IntentClassifier:
    def __init__(self, chatbot_data: ChatbotData):
        self.data = chatbot_data
        self.preprocessor = TextPreprocessor()
        self.vectorizer = TfidfVectorizer()
        self._train_classifier()
    
    def _train_classifier(self):
        """Prepare training data for intent classification"""
        self.training_data = []
        self.labels = []
        
        for intent, data in self.data.intents.items():
            for pattern in data["patterns"]:
                processed_pattern = self.preprocessor.preprocess_text(pattern)
                self.training_data.append(processed_pattern)
                self.labels.append(intent)
        
        # Fit TF-IDF vectorizer
        if self.training_data:
            self.tfidf_matrix = self.vectorizer.fit_transform(self.training_data)
    
    def classify_intent(self, text: str) -> Tuple[str, float]:
        """Classify user intent using similarity matching"""
        processed_text = self.preprocessor.preprocess_text(text)
        
        if not self.training_data:
            return "unknown", 0.0
        
        # Transform input text
        text_vector = self.vectorizer.transform([processed_text])
        
        # Calculate similarity with all training examples
        similarities = cosine_similarity(text_vector, self.tfidf_matrix)
        max_similarity = similarities.max()
        best_match_idx = similarities.argmax()
        
        # Threshold for confidence
        if max_similarity > 0.3:
            return self.labels[best_match_idx], max_similarity
        else:
            return "unknown", max_similarity

class CustomerSupportChatbot:
    def __init__(self):
        self.data = ChatbotData()
        self.intent_classifier = IntentClassifier(self.data)
        self.preprocessor = TextPreprocessor()
        self.conversation_history = []
        
    def process_query(self, user_input: str) -> str:
        """Main method to process user query and generate response"""
        intent, confidence = self.intent_classifier.classify_intent(user_input)
        entities = self.preprocessor.extract_entities(user_input)
        
        # Log the interaction
        self.conversation_history.append({
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'user_input': user_input,
            'intent': intent,
            'confidence': round(confidence, 2),
            'entities': entities,
            'response': None
        })
        
        if intent == "unknown" or confidence < 0.3:
            response = self._get_fallback_response()
        else:
            response = self._generate_response(intent, entities)
        
        # Update conversation history with response
        self.conversation_history[-1]['response'] = response
        
        return response
    
    def _generate_response(self, intent: str, entities: Dict) -> str:
        """Generate appropriate response based on intent and entities"""
        intent_data = self.data.intents.get(intent)
        if not intent_data:
            return self._get_fallback_response()
        
        # Select random response template
        response_template = random.choice(intent_data["responses"])
        
        # Handle specific intents with dynamic data
        if intent == "order_status":
            return self._handle_order_status(response_template, entities)
        elif intent == "product_info":
            return self._handle_product_info(response_template, entities)
        else:
            return response_template
    
    def _handle_order_status(self, template: str, entities: Dict) -> str:
        """Handle order status queries"""
        order_numbers = entities.get("order_number", [])
        
        if not order_numbers:
            return "Please provide your order number so I can check the status for you."
        
        order_number = order_numbers[0]
        order_info = self.data.get_order_status(order_number)
        
        if order_info["status"] == "not found":
            return f"Sorry, I couldn't find order #{order_number}. Please check the order number and try again."
        
        return template.format(
            order_number=order_number, 
            status=order_info["status"],
            estimated_delivery=order_info["estimated_delivery"]
        )
    
    def _handle_product_info(self, template: str, entities: Dict) -> str:
        """Handle product information queries"""
        product_names = entities.get("product_name", [])
        
        if product_names:
            product_name = product_names[0].lower()
            product_info = self.data.get_product_info(product_name)
            return f"For {product_name}: {product_info}"
        
        return template
    
    def _get_fallback_response(self) -> str:
        """Provide fallback response for unknown queries"""
        fallback_responses = [
            "I'm not sure I understand. Could you rephrase your question?",
            "I'm here to help with order status, returns, delivery times, and product information.",
            "Could you provide more details about your query?",
            "I specialize in order tracking, return policies, and product FAQs. How can I help?"
        ]
        return random.choice(fallback_responses)
    
    def get_conversation_history(self) -> List[Dict]:
        """Get the complete conversation history"""
        return self.conversation_history
    
    def display_conversation_stats(self):
        """Display statistics about the conversation"""
        if not self.conversation_history:
            print("No conversation history available.")
            return
        
        print(f"\nConversation Statistics:")
        print(f"Total exchanges: {len(self.conversation_history)}")
        
        intent_counts = defaultdict(int)
        for exchange in self.conversation_history:
            intent_counts[exchange['intent']] += 1
        
        for intent, count in intent_counts.items():
            print(f"  {intent}: {count}")

In [6]:
# Enhanced version with basic sentiment analysis
class AdvancedChatbot(CustomerSupportChatbot):
    def __init__(self):
        super().__init__()
        self.negative_keywords = ['angry', 'upset', 'frustrated', 'terrible', 'horrible', 'awful', 'bad', 'poor', 'worst']
        self.positive_keywords = ['great', 'excellent', 'awesome', 'good', 'wonderful', 'fantastic', 'amazing']
        
    def analyze_sentiment(self, text: str) -> str:
        """Basic sentiment analysis using keyword matching"""
        text_lower = text.lower()
        
        positive_count = sum(1 for word in self.positive_keywords if word in text_lower)
        negative_count = sum(1 for word in self.negative_keywords if word in text_lower)
        
        if negative_count > positive_count:
            return "negative"
        elif positive_count > negative_count:
            return "positive"
        else:
            return "neutral"
    
    def process_query(self, user_input: str) -> str:
        sentiment = self.analyze_sentiment(user_input)
        base_response = super().process_query(user_input)
        
        # Add empathetic responses for negative sentiment
        if sentiment == "negative":
            empathetic_prefixes = [
                "I understand this might be frustrating. ",
                "I apologize for any inconvenience. ",
                "I'm sorry to hear you're having issues. ",
                "Let me help resolve this for you. "
            ]
            return random.choice(empathetic_prefixes) + base_response.lower()
        elif sentiment == "positive":
            positive_prefixes = [
                "Great to hear! ",
                "I'm glad you're happy with our service! ",
                "Wonderful! "
            ]
            return random.choice(positive_prefixes) + base_response
        
        return base_response

# Example usage of advanced features
def demo_advanced_chatbot():
    advanced_bot = AdvancedChatbot()
    
    test_scenarios = [
        "My order is late and I'm really frustrated!",
        "Great service, thank you so much!",
        "Where is my order 12345?",
        "This is terrible service, my order never arrived!"
    ]
    
    print("ADVANCED CHATBOT DEMO WITH SENTIMENT ANALYSIS")
    print("=" * 60)
    
    for query in test_scenarios:
        print(f"\nUser: {query}")
        response = advanced_bot.process_query(query)
        print(f"Bot: {response}")
        print("-" * 60)

In [8]:
def run_chatbot_cli():
    """Run the chatbot in command line interface"""
    chatbot = CustomerSupportChatbot()
    
    print("=" * 60)
    print("CUSTOMER SUPPORT CHATBOT")
    print("=" * 60)
    print("I can help you with:")
    print("• Order status and tracking")
    print("• Return policies and refunds") 
    print("• Delivery times and shipping")
    print("• Product information and features")
    print("• Order cancellations")
    print("• Contact information")
    print("\nType 'quit', 'exit', or 'bye' to end the conversation")
    print("Type 'history' to see conversation history")
    print("Type 'stats' to see conversation statistics")
    print("-" * 60)
    
    while True:
        try:
            user_input = input("\nYou: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                print("Bot: Thank you for contacting customer support. Have a great day!")
                break
            
            if user_input.lower() == 'history':
                history = chatbot.get_conversation_history()
                if history:
                    print("\nConversation History:")
                    for i, exchange in enumerate(history, 1):
                        print(f"{i}. You: {exchange['user_input']}")
                        print(f"   Bot: {exchange['response']}")
                        print(f"   [Intent: {exchange['intent']}, Confidence: {exchange['confidence']}]")
                else:
                    print("No conversation history yet.")
                continue
                
            if user_input.lower() == 'stats':
                chatbot.display_conversation_stats()
                continue
            
            if not user_input:
                continue
                
            response = chatbot.process_query(user_input)
            print(f"Bot: {response}")
            
        except KeyboardInterrupt:
            print("\nBot: Session ended. Thank you for using our customer support!")
            break
        except Exception as e:
            print(f"Bot: I encountered an error. Please try again.")

def run_chatbot_demo():
    """Run a demo with predefined test cases"""
    chatbot = CustomerSupportChatbot()
    
    test_queries = [
        "Hello",
        "Where is my order 12345?",
        "How can I return a product?",
        "What's the delivery time?",
        "Does the phone support fast charging?",
        "I want to cancel my order",
        "Can I talk to a human?",
        "Thank you",
        "What about order 99999?",
        "Tell me about laptops",
        "What's the status of order 67890?",
        "How do I return a defective item?"
    ]
    
    print("CHATBOT DEMO - CUSTOMER SUPPORT")
    print("=" * 50)
    
    for query in test_queries:
        print(f"\nUser: {query}")
        response = chatbot.process_query(query)
        print(f"Bot: {response}")
        print("-" * 50)

def main():
    """Main function to run the chatbot"""
    print("Customer Support Chatbot")
    print("1. Interactive Chat Mode")
    print("2. Demo Mode")
    
    while True:
        choice = input("\nEnter your choice (1 or 2): ").strip()
        
        if choice == "1":
            run_chatbot_cli()
            break
        elif choice == "2":
            run_chatbot_demo()
            break
        else:
            print("Invalid choice. Please enter 1 or 2.")

if __name__ == "__main__":
    main()

Customer Support Chatbot
1. Interactive Chat Mode
2. Demo Mode



Enter your choice (1 or 2):  1


CUSTOMER SUPPORT CHATBOT
I can help you with:
• Order status and tracking
• Return policies and refunds
• Delivery times and shipping
• Product information and features
• Order cancellations
• Contact information

Type 'quit', 'exit', or 'bye' to end the conversation
Type 'history' to see conversation history
Type 'stats' to see conversation statistics
------------------------------------------------------------



You:  Delivery time


Bot: Standard delivery takes 3-5 business days, express takes 1-2 days.



You:  Order Cancellation


Bot: Orders can be cancelled within 1 hour via your account dashboard. After that, contact support.



You:  Exit


Bot: Thank you for contacting customer support. Have a great day!
